In this notebook, we process the matrices needed in Proving Property (T) for $SL_3(\mathbb{Z})$ via semidefinite optimization.

In [6]:
import numpy as np
import itertools
from scipy.io import savemat
from functools import reduce
import os
np.version.version
length_of_Mprime = 121
width_of_Mprime = 121

In [8]:
# We open the matrix containing coefficients of Delta^2 - 0.2805 Delta
with open('~/Desktop/BMatrixData/DeltaCoeffs.bin','rb') as f:
    depth=np.fromfile(f,dtype=np.dtype('int32'),count=1)
    dims =np.fromfile(f,dtype=np.dtype('int32'),count=depth[0])
    data =np.reshape(np.fromfile(f,dtype=np.dtype('float64'), count=reduce(lambda x,y:x*y,dims)),dims)
arr_list = []
counter = 0
counter2 = 0
constraints = set()
unique_group_elements = set()
os.makedirs("~/Desktop/MatlabMatrices/")
os.makedirs("~/Desktop/MatlabMatrices/PosNonredundant")
os.makedirs("~/Desktop/MatlabMatrices/UniqueGroupElts")
for i in range(1, length_of_Mprime+1):
    for j in range(1, width_of_Mprime+1):
        # We construct the matrix B_{i, j} defined entrywise at i', j' 
        # by B_{i, j}(i', j') = 1 if b_i^*b_j = b_{i'}^*b_{j'}
        # 0 otherwise
        with open('~/Desktop/BMatrixData/Positions'+str(i)+'_'+str(j)+'.bin','rb') as f:
            depth=np.fromfile(f,dtype=np.dtype('int32'),count=1)
            dims =np.fromfile(f,dtype=np.dtype('int32'),count=depth[0])
            data =np.reshape(np.fromfile(f,dtype=np.dtype('int32'), count=reduce(lambda x,y:x*y,dims)),dims)
        arr = np.zeros((length_of_Mprime, length_of_Mprime))
        for pair in data:
            arr[pair[0]-1, pair[1]-1] = 1
        as_tuple = tuple(map(tuple, (arr + np.transpose(arr))/2))
        # We add the constraint Tr(B_{i, j}X) = coefficient of b_i^*b_j in Delta^2 -\lambda Delta
        # In this first if statement, we notice that this constraint is the same as 
        # Tr((B_{i, j} + B_{i, j}^T)/2 X) = coefficient of b_i^*b_j in Delta^2 -\lambda Delta
        # and we try to not repeat any of the constraints
        # In the second if statement, we keep redundant constraints of the form
        # Tr((B_{i, j} + B_{i, j}^T)/2 X) = coefficient of b_i^*b_j in Delta^2 -\lambda Delta
        # As long as no two B_{i, j} are the same.
        if as_tuple not in constraints:
            mdic = {"a": (arr + np.transpose(arr))/2, "b": deltadata[(i-1)][(j-1)]}
            savemat("~/Desktop/MatlabMatrices/PosNonredundant/Pos" + str(counter)+".mat", mdic)
            constraints.add(as_tuple)
            counter += 1
        not_necessarily_symmetric = tuple(map(tuple, arr))
        if not_necessarily_symmetric not in unique_group_elements:
            mdic = {"a": (arr + np.transpose(arr))/2, "b": deltadata[(i-1)][(j-1)]}
            savemat("~/Desktop/MatlabMatrices/UniqueGroupElts/Pos" + str(counter2)+".mat", mdic)
            unique_group_elements.add(not_necessarily_symmetric)
            counter2 += 1